In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

In [ ]:
# Storing the recipes information into a pandas DataFrame titled 'recipes'
recipes = pd.read_csv('recipes.csv', engine='python', on_bad_lines='skip')

In [ ]:
# Storing the reviews information into a pandas DataFrame titled 'reviews'
reviews = pd.read_csv('reviews.csv', engine='python', on_bad_lines='skip')

# ⚡ Data pre-processing

In [ ]:
# Truncating the DateSubmitted and DateModified columns
reviews=reviews[['ReviewId','RecipeId','AuthorId','AuthorName','Rating','Review']]

In [ ]:
# Function for counting Null values 
def countNullValues(col):
  c = 0
  for i in col:
    if(i != i):
      c+=1
  return c

In [ ]:
# Counting null values in the reviews dataframe
c1 = countNullValues(reviews.ReviewId)
c2 = countNullValues(reviews.RecipeId) 
c3 = countNullValues(reviews.AuthorId)
c4 = countNullValues(reviews.AuthorName) 
c5 = countNullValues(reviews.Rating)
c6 = countNullValues(reviews.Review) 

In [ ]:
# Function for replacing Null Values
def replaceNullValue(column):
  j = 0
  for i in column:
    if(str(i) == 'nan'):
      column[j] = 'NA'
    j+=1
  return column

In [ ]:
replaceNullValue(reviews.Review)

In [ ]:
# Checking the column labels in reviews
reviews.columns, reviews.shape

In [ ]:
# Printing a summary of the data in reviews data
# .describe() gives the statistics of the numerical data 
reviews.describe()

In [ ]:
# Checking the Rating distribution in the range 0-5 for the Data given 
import seaborn as sns
with sns.axes_style('white'):
    g = sns.catplot(x="Rating", data=reviews, aspect=2.0,kind='count')
    g.set_ylabels("Total number of ratings")

In [ ]:
print("\nTotal no of ratings :", reviews.shape[0])
print("Total No of authors   :", len(np.unique(reviews.AuthorId)))
print("Total No of recipes  :", len(np.unique(reviews.RecipeId)))

# ⚡ Item-based Collaborative based Recommender

In [ ]:
# Setting rating count threshold
threshold = 30
counts = reviews['RecipeId'].value_counts()
valid_recipes = counts[counts >= threshold].index
valid_recipes

In [ ]:
filtered_ratings = reviews[reviews['RecipeId'].isin(valid_recipes)]
filtered_ratings.shape

In [ ]:
# Reset index value
filtered_ratings.reset_index(inplace = True, drop = True)
filtered_ratings.head()

In [ ]:
# Get user rating matrix
matrix = filtered_ratings.pivot(index = 'AuthorId', columns ='RecipeId', values = 'Rating').fillna(0)
matrix

In [ ]:
with open('filtered_ratings.pkl', 'wb') as f:
    pickle.dump(filtered_ratings, f)

In [ ]:
# testing input
users_rating = matrix[56]
users_rating.sort_values(ascending=False).head(20)

In [ ]:
similar_recipes = matrix.corrwith(users_rating)
similar_recipes

In [ ]:
similar_recipes = pd.DataFrame(similar_recipes,columns=['correlation'])
similar_recipes.head(10)

In [ ]:
similar_recipes.sort_values(by='correlation', ascending=False).head(20)

# Weighted Rating calculate

In [ ]:
# Calculating the ratings given per recipe
RatingCounts = filtered_ratings.groupby('RecipeId')['Rating'].count()
RatingCounts.sort_values(ascending=False, inplace=True)

In [ ]:
m = threshold

In [ ]:
# Creating and merging smaller recipes dataframe with individual rating counts for a recipe
#recipesSubDataFrame = recipes.iloc[:9000]
weightedPopularRecipes = recipes.merge(RatingCounts, on='RecipeId')

In [ ]:
RatingAverages = weightedPopularRecipes[weightedPopularRecipes['AggregatedRating'].notnull()]['AggregatedRating'].astype('int')
C = RatingAverages.mean()
C

In [ ]:
# Columns selected to appear in shortlisted dataframe are RecipeId, Name, RecipeCategory, AggregatedRating, Rating, and ReviewCount
qualifiedRecipes = weightedPopularRecipes[(weightedPopularRecipes['Rating'] >= threshold) & (weightedPopularRecipes['Rating'].notnull()) & (weightedPopularRecipes['AggregatedRating'].notnull())][['RecipeId', 'Name', 'RecipeCategory', 'AggregatedRating', 'Rating', 'ReviewCount']]

# Setting the datatype of 'Rating' and 'AggregatedRating' to int
qualifiedRecipes['Rating'] = qualifiedRecipes['Rating'].astype('int')
qualifiedRecipes['AggregatedRating'] = qualifiedRecipes['AggregatedRating'].astype('int') 

In [ ]:
# Function that defines the mathematical formula for the weighted rating method
def weightedRating(x):
    v = x['Rating']
    R = x['AggregatedRating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Applying the WR function
qualifiedRecipes['score'] = qualifiedRecipes.apply(weightedRating, axis=1)
#qualifiedRecipes.loc[qualifiedRecipes["RecipeId"] == 56]
qualifiedRecipes.head()
names = qualifiedRecipes[["RecipeId","Name"]]
names

In [ ]:
with open('qualifiedRecipesCollaborative.pkl', 'wb') as f:
    pickle.dump(qualifiedRecipes, f)

In [ ]:
# Convert into same dataframe format 
df_rating = pd.DataFrame(qualifiedRecipes.groupby(by = ['RecipeId'])['score'].agg('mean')) 
df_rating.head(10)

In [ ]:
result_recipes = similar_recipes.join(df_rating['score'] ).sort_values(by='correlation', ascending=False)
result_recipes = result_recipes[result_recipes['score'] > 4].sort_values(by = 'correlation', ascending = False)
result_recipes

In [ ]:
#reset = result_recipes.reset_index()
#reset

In [ ]:
def getCollaborativeRecommendations(id):
    
    if id in matrix.columns:
        # take the column of data for the recipe
        users_rating = matrix[id]
        # find correlation with the recipe
        similar_recipes = matrix.corrwith(users_rating)
        # convert into data frame
        similar_recipes = pd.DataFrame(similar_recipes,columns=['correlation'])
        # join the correlation with weighted rating score atleast higher than 4
        result_recipes = similar_recipes.join(df_rating['score']).sort_values(by='correlation', ascending=False)
        result_recipes = result_recipes[result_recipes['score'] > 4].sort_values(by = 'correlation', ascending = False)
        
        # selecting the top 5
        recipeIndices = result_recipes.iloc[1:6].index.tolist()
        # return top 5 recipes
        return names[names['RecipeId'].isin(recipeIndices)]
    
    else:
        return -1
        

In [ ]:
getCollaborativeRecommendations(56)

In [ ]:
#names.loc[names['RecipeId']==recipeIndices]

In [ ]:
#index = [87421,101568,229104,89030]
#test = names[names['RecipeId'].isin(index)]
#test